In [1]:
import numpy as np
import pandas as pd
import os
import sys
import funcs as f
import xarray 

In [2]:
outpath       = "/Volumes/Baby-Orange/XPIA-NCAR/processed/"
datapath      = "/Volumes/Baby-Orange/XPIA-NCAR/rawest/"

In [8]:
df_zi       = pd.read_csv("/Volumes/Baby-Orange/XPIA-NCAR/processed/ziL_all_days.csv", header=None, index_col=[0], parse_dates=True)

In [9]:
sims=["LES","vles","mynn","ysu","sh"]

datetimes = []
year      = 2015
month     = 3
days      = [20, 21, 28, 29, 30] #20
hours     = np.arange(14,24,1)
minutes   = np.arange(0,60,10)

it = 0
for day in days:
    for hour in hours:
        for minute in minutes:
            sys.stdout.write('\r'+str(it))
            datetime = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=minute)
            datetimes.append(datetime) 
            it += 1

299

In [10]:
# how much distance should we consider along x and y?
box_side = 15000 # 15 km

les_npts= int(box_side/25.0)
tot_pts = 1200
les_pt1 = int((tot_pts/2)-les_npts/2)
les_pt2 = les_pt1 + les_npts
les_pts = np.arange(les_pt1,les_pt2,1)
les_pts = [int(a) for a in les_pts]

# MUST FIX THESE
gz_npts = int(box_side/333.0)
tot_pts = 90
gz_pt1  = int((tot_pts/2)-int(gz_npts/2))
gz_pt2  = gz_pt1 + gz_npts
gz_pts  = np.arange(gz_pt1,gz_pt2,1)
# sim = "LES"
print("will consider RLES points between {0} and {1} ({2} pts)".format(les_pt1, les_pt2,len(les_pts)))
print("will consider VLES, YSU, SH, MYNN points between {0} and {1} ({2} pts)".format(gz_pt1, gz_pt2,len(gz_pts)))

will consider RLES points between 300 and 900 (600 pts)
will consider VLES, YSU, SH, MYNN points between 23 and 68 (45 pts)


In [13]:
convective_layer_temps = pd.DataFrame(index=datetimes,columns=sims)   

for datetime in datetimes:

    zi           = df_zi.loc[datetime].values[0]   
    datetime_str = "{0:%Y-%m-%d_%H:%M}".format(datetime)

    sys.stdout.write('\r'+datetime_str)
    
    for sim in sims:        

        pts = gz_pts if sim!="LES" else les_pts                
        
        sim_prefix       = sim+"_25m" if sim=="LES" else sim
        fpath            = os.path.join(datapath,"WRF_{0}_3D_{1}_plus_filtered.nc".format(sim_prefix,datetime_str))    
        data             = xarray.open_dataset(fpath)

        z         = data['z'].copy()
        zmean     = np.median(z.values,axis=(1,2))
        z_zi      = zmean/zi    
        k         = np.argmin(np.abs(z_zi-1.0))                    
        
        layer_temp = data["theta"].isel(west_east=pts).isel(south_north=pts)
        layer_temp = layer_temp.values
        layer_temp = layer_temp[k,...].copy()
        convective_layer_temps.loc[datetime,sim] = np.median(layer_temp)

2015-03-30_23:50

# save

In [16]:
days = [20,21,28,29,30]

In [18]:
for day in days:
    daydata = convective_layer_temps.loc['2015-03-{0}'.format(day)].copy()
    #print (daydata.head())
    #print (daydata.tail())
    fout = os.path.join(outpath,"T_LAYER_SPATIAL_AVG_15km_by_15km_March{0}.csv".format(day))
    print(fout)    
    daydata.to_csv(fout)

/Volumes/Baby-Orange/XPIA-NCAR/processed/T_LAYER_SPATIAL_AVG_15km_by_15km_March20.csv
/Volumes/Baby-Orange/XPIA-NCAR/processed/T_LAYER_SPATIAL_AVG_15km_by_15km_March21.csv
/Volumes/Baby-Orange/XPIA-NCAR/processed/T_LAYER_SPATIAL_AVG_15km_by_15km_March28.csv
/Volumes/Baby-Orange/XPIA-NCAR/processed/T_LAYER_SPATIAL_AVG_15km_by_15km_March29.csv
/Volumes/Baby-Orange/XPIA-NCAR/processed/T_LAYER_SPATIAL_AVG_15km_by_15km_March30.csv
